# 16_Defining_One_to_Many_Relationships

In this guide, we will learn how to define a **One-to-Many** relationship in Spring Data JPA using the example of a `User` who can have multiple `Address`es.

---

## 🧑‍💻 Domain Logic: One User ➡️ Many Addresses

### 🧱 Entities Overview:

* A **User** can have **many** addresses.
* An **Address** belongs to **one** User.

---

## ✅ Step-by-step Code Explanation

### 🧑 `User.java`

```java
@Entity
@Table(name = "users")
@ToString
@Builder
@Setter
@Getter
@AllArgsConstructor
@NoArgsConstructor
public class User {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    @Column(name = "id")
    private Long id;

    @Column(name = "name")
    private String name;

    @Column(name = "email")
    private String email;

    @Column(name = "password")
    private String password;

    @OneToMany(mappedBy = "user") // 🔁 One user -> many addresses
    @Builder.Default
    private List<Address> addresses = new ArrayList<>();

    public void addAddress(Address address) {
        this.addresses.add(address);
        address.setUser(this);
    }

    public void removeAddress(Address address) {
        this.addresses.remove(address);
        address.setUser(null);
    }
}
```

### 🏠 `Address.java`

```java
@Entity
@Table(name = "Addresses")
@ToString
@Builder
@Getter
@Setter
@AllArgsConstructor
@NoArgsConstructor
public class Address {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    @Column(name = "id")
    private Long id;

    @Column(name = "street")
    private String street;

    @Column(name = "city")
    private String city;

    @Column(name = "zip")
    private String zip;

    @Column(name = "state")
    private String state;

    @ManyToOne // 🔁 Many addresses -> one user
    @JoinColumn(name = "user_id") // 🔗 Foreign key in address table
    @ToString.Exclude // 🚫 Avoid circular reference in toString()
    private User user;
}
```

---

## 🧠 Key Annotations Explained

### 🔄 `@OneToMany(mappedBy = "user")`

* Placed in the **parent** entity (`User`).
* Tells JPA that the `Address` entity has a field named `user` that owns the relationship.
* `mappedBy` means **this side is inverse** (does not own the relationship).

### 📌 `@ManyToOne`

* Placed in the **child** entity (`Address`).
* Indicates that each `Address` is associated with **one** `User`.

### 🔗 `@JoinColumn(name = "user_id")`

* Specifies the **foreign key** column in the `addresses` table.
* This column links each address to a user.

### 🚫 `@ToString.Exclude`

* Prevents infinite recursion in the `toString()` method when both entities reference each other.

---

## 🔍 Why `mappedBy` is in `User`, not `Address`?

Because the foreign key (`user_id`) lives in the **Address** table, `Address` is the **owner** of the relationship. So:

* `User` declares the relationship as `@OneToMany(mappedBy = "user")` — this side is the inverse.
* `Address` uses `@ManyToOne` with `@JoinColumn` — this side owns the relationship.

---

## 🧪 Testing the Relationship

### 🧾 `StoreApplication.java`

```java
@SpringBootApplication
public class StoreApplication {

    public static void main(String[] args) {
        ApplicationContext context = SpringApplication.run(StoreApplication.class, args);

        var user = User.builder()
                .name("John")
                .email("john@gmail.com")
                .password("1234")
                .build();

        var address = Address.builder()
                .street("street")
                .city("city")
                .state("state")
                .zip("zip")
                .build();

        user.addAddress(address); // 👈 Setting both sides of the relationship

        System.out.println(user); // 🔁 Prints user with address (no infinite loop thanks to @ToString.Exclude)
    }
}
```

✅ Now you’ve built a clean, bi-directional One-to-Many relationship in JPA with Lombok and explained each concept step-by-step! 🚀
